## Movie Recommendation System

In [1]:
import pandas as pd

movies=pd.read_csv("movies.csv")

In [2]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


# Title Cleaning Function

This code preprocesses movie titles by removing special characters, keeping only alphanumeric characters and spaces.

## How It Works

1. **Define Cleaning Function**: Uses regex to remove all non-alphanumeric characters (except spaces)
2. **Apply to Dataset**: Creates a new column `clean_title` by applying the function to all movie titles

## Purpose

- Standardizes titles for better matching (e.g., "Spider-Man" becomes "SpiderMan")
- Removes punctuation, symbols, and special characters
- Improves TF-IDF vectorization accuracy by reducing noise

**Example**: "The Matrix: Reloaded" → "The Matrix Reloaded"

In [3]:
import re

def clean_title(title):
    return re.sub("[^a-zA-Z0-9 ]","",title)

In [5]:
movies["clean_title"]=movies["title"].apply(clean_title)

In [6]:
movies

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995
...,...,...,...,...
62418,209157,We (2018),Drama,We 2018
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul 2001
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems 2018
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing 2001


# TF-IDF Vectorization Explained

This code uses scikit-learn's `TfidfVectorizer` to convert the cleaned movie titles into a sparse matrix of numerical features. This numerical matrix is essential for machine learning tasks like similarity matching or recommendation systems.

| Component | Purpose | Description |
|-----------|---------|-------------|
| `TfidfVectorizer` | The Tool | Calculates the Term Frequency-Inverse Document Frequency for words, weighting important (rare) words higher. |
| `ngram_range=(1,2)` | Feature Scope | Configures the vectorizer to consider both single words (unigrams) and sequential pairs of words (bigrams) as features. |
| `fit_transform()` | Execution | First, it builds the vocabulary from all titles (fit), then calculates and returns the TF-IDF matrix (transform). |
| `tfidf` | The Output | A sparse matrix where each row represents a movie and each column represents a word/bigram feature, populated with TF-IDF scores. |

## Example: How TF-IDF Works

The TF-IDF score for a word in a document is the product of two measures: **Term Frequency (TF)** and **Inverse Document Frequency (IDF)**.

$$\text{TFIDF}(t, d) = \text{TF}(t, d) \times \text{IDF}(t)$$

### 1. Term Frequency (TF)

This measures how frequently a term $t$ appears in a document $d$. (The `TfidfVectorizer` often uses a normalized/smoothed version of this.)

### 2. Inverse Document Frequency (IDF)

This measures how important a term is by weighing down common terms (like "The" or "A") and boosting rarer, more distinctive terms (like "Jedi" or "Inception").

$$\text{IDF}(t) = \log\left(\frac{\text{Total number of documents}}{\text{Number of documents containing term } t}\right)$$

## Calculation Example

Imagine we have two clean titles:

| ID | Title (d) |
|----|-----------|
| 1 | The Cat in the Hat |
| 2 | Cat on a Hot Tin Roof |

| Term (t) | TF in Title 1 | Documents with Term | Total Documents | IDF Score |
|----------|---------------|---------------------|-----------------|-----------|
| The | $2 / 5 = 0.4$ | 1 | 2 | $\log(2/1) \approx 0.30$ |
| Cat | $1 / 5 = 0.2$ | 2 | 2 | $\log(2/2) = 0.00$ |
| Hat | $1 / 5 = 0.2$ | 1 | 2 | $\log(2/1) \approx 0.30$ |

**The TF-IDF Score for "Hat" in Title 1 is:**

$$\text{TFIDF}(\text{Hat, Title 1}) = \text{TF} \times \text{IDF} = 0.2 \times 0.30 \approx 0.06$$

**The TF-IDF Score for the common word "The" in Title 1 is:**

$$\text{TFIDF}(\text{The, Title 1}) = \text{TF} \times \text{IDF} = 0.4 \times 0.30 \approx 0.12$$

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer=TfidfVectorizer(ngram_range=(1,2))

tfidf=vectorizer.fit_transform(movies["clean_title"])

# Movie Search Function Using Cosine Similarity

## How It Works

1. **Clean Input**: Preprocesses the search query
2. **Vectorize Query**: Converts title into a TF-IDF vector
3. **Calculate Similarity**: Computes cosine similarity between query and all movies
4. **Find Top Matches**: Gets indices of 5 highest similarity scores
5. **Retrieve Results**: Returns the top 5 matching movies in descending order

## Cosine Similarity

Measures similarity between two vectors, ranging from 0 (dissimilar) to 1 (identical).

$$\text{cosine similarity}(A, B) = \frac{A \cdot B}{\|A\| \|B\|}$$

**Example**: Searching "The Dark Knight" returns Batman-related films with highest similarity scores.

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(title):
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    result = movies.iloc[indices][::-1]
    return result

# Interactive Movie Search Widget

This code creates an interactive search interface using Jupyter widgets that displays matching movies as you type.

## Components

| Component | Purpose |
|-----------|---------|
| `widgets.Text()` | Creates an input box for typing movie titles |
| `widgets.Output()` | Creates a display area for search results |
| `on_type()` | Callback function triggered when input changes |
| `observe()` | Monitors the input widget for value changes |

## How It Works

1. User types in the text input box
2. The `on_type()` function is triggered on every keystroke
3. If the query is longer than 5 characters, it calls `search()`
4. Results are displayed in the output widget below the input box
5. Previous results are cleared before showing new ones

**Note**: The widget provides real-time search as you type, updating results dynamically.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

movies_input=widgets.Text(
    value="Tere Naam",
    description='Movie Title:',
    disabled=False,
)

movies_list=widgets.Output()

def on_type(data):
    with movies_list:
        movies_list.clear_output()
        title=data["new"]
        if len(title)>5:
            display(search(title))

movies_input.observe(on_type, names="value")

display(movies_input,movies_list)

Text(value='Toy Story', description='Movie Title:')

Output()

In [10]:
ratings=pd.read_csv("ratings.csv")

In [11]:
ratings

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


In [13]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

# Find Similar Movies Function

This function recommends movies based on collaborative filtering by analyzing users who rated a given movie highly.

## How It Works

| Step | Description |
|------|-------------|
| 1. Find Similar Users | Identifies users who rated the target movie above 4 stars |
| 2. Get Their Recommendations | Finds other movies these users rated highly (>4 stars) |
| 3. Calculate Popularity | Computes what percentage of similar users liked each movie (filters out movies liked by <10% of users) |
| 4. Get All Users Baseline | Calculates what percentage of all users liked these same movies |
| 5. Compute Score | Divides similar user percentage by all user percentage to find movies that are disproportionately popular among similar users |
| 6. Return Top 10 | Sorts by score and returns the top 10 recommendations with title and genre information |

## The Scoring Formula

$$\text{score} = \frac{\text{similar user popularity}}{\text{all user popularity}}$$

**Higher scores** indicate movies that are particularly popular among users who liked the target movie, but not as popular overall (hidden gems).

## Example

If you search for similar movies to **"Toy Story"**:
- Finds users who rated Toy Story >4 stars
- Discovers what else they loved
- Returns movies like "Toy Story 2", "Finding Nemo", or other Pixar films that this group particularly enjoys

In [17]:
def find_similar_movies(movie_id):
    similar_users=ratings[(ratings["movieId"]==movie_id) & (ratings["rating"]>4)]["userId"].unique()
    similar_users_recs=ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"]>4)]["movieId"]

    similar_users_recs=similar_users_recs.value_counts() / len(similar_users)
    similar_users_recs=similar_users_recs[similar_users_recs > .10]

    all_users=ratings[(ratings["movieId"].isin(similar_users_recs.index)) & (ratings["rating"]>4)]
    all_users_recs=all_users["movieId"].value_counts() / len(all_users["userId"].unique())

    rec_percentages=pd.concat([similar_users_recs,all_users_recs],axis=1)
    rec_percentages.columns=["similar","all"]

    rec_percentages["score"]=rec_percentages["similar"]/rec_percentages["all"]

    rec_percentages=rec_percentages.sort_values("score",ascending=False)
    return rec_percentages.head(10).merge(movies,left_index=True,right_on="movieId")[["score","title","genres"]]

In [18]:
import ipywidgets as widgets
from IPython.display import display

movie_name_input = widgets.Text(
    value='Tere Naam',
    description='Movie Title:',
    disabled=False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title)
            movie_id = results.iloc[0]["movieId"]
            display(find_similar_movies(movie_id))

movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)

Text(value='Tere Naam', description='Movie Title:')

Output()